In [1]:
!pip install pyspark

In [2]:
!pip install findspark

In [2]:
import pyspark
from pyspark.sql import SparkSession

In [3]:
import findspark
findspark.init()

In [4]:
spark = SparkSession.builder.appName("PySparkExample").getOrCreate()


In [21]:
df = spark.read.format("csv").option("header","true").option("inferSchema","true").option("mode","DROPMALFORMED").load("C:/Users/oseda/Documents/Cloud Assignment/dataset_olympics.csv")

In [22]:
df.show()

+---+--------------------+---+----+------+------+--------------+---+-----------+----+------+-----------+--------------------+--------------------+-----+
| ID|                Name|Sex| Age|Height|Weight|          Team|NOC|      Games|Year|Season|       City|               Sport|               Event|Medal|
+---+--------------------+---+----+------+------+--------------+---+-----------+----+------+-----------+--------------------+--------------------+-----+
|  1|           A Dijiang|  M|24.0| 180.0|  80.0|         China|CHN|1992 Summer|1992|Summer|  Barcelona|          Basketball|Basketball Men's ...| NULL|
|  2|            A Lamusi|  M|23.0| 170.0|  60.0|         China|CHN|2012 Summer|2012|Summer|     London|                Judo|Judo Men's Extra-...| NULL|
|  3| Gunnar Nielsen Aaby|  M|24.0|  NULL|  NULL|       Denmark|DEN|1920 Summer|1920|Summer|  Antwerpen|            Football|Football Men's Fo...| NULL|
|  4|Edgar Lindenau Aabye|  M|34.0|  NULL|  NULL|Denmark/Sweden|DEN|1900 Summer|19

In [40]:
df = df.fillna(value="None",subset=["Medal"])

In [41]:
df = df.fillna(value=0.0,subset=["Height","Weight"])

In [42]:
df = df.drop("NOC")

In [44]:
df.show()

+---+--------------------+---+----+------+------+--------------+-----------+----+------+-----------+--------------------+--------------------+-----+
| ID|                Name|Sex| Age|Height|Weight|          Team|      Games|Year|Season|       City|               Sport|               Event|Medal|
+---+--------------------+---+----+------+------+--------------+-----------+----+------+-----------+--------------------+--------------------+-----+
|  1|           A Dijiang|  M|24.0| 180.0|  80.0|         China|1992 Summer|1992|Summer|  Barcelona|          Basketball|Basketball Men's ...| None|
|  2|            A Lamusi|  M|23.0| 170.0|  60.0|         China|2012 Summer|2012|Summer|     London|                Judo|Judo Men's Extra-...| None|
|  3| Gunnar Nielsen Aaby|  M|24.0|  NULL|   0.0|       Denmark|1920 Summer|1920|Summer|  Antwerpen|            Football|Football Men's Fo...| None|
|  4|Edgar Lindenau Aabye|  M|34.0|  NULL|   0.0|Denmark/Sweden|1900 Summer|1900|Summer|      Paris|      